**Config**

In [38]:
import torch

# edit the config
device = torch.device('cpu')
dataset_name = 'mgif' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = './input/4_(2).png'
driving_video_path = './input/3.mp4'
output_video_path = './normal.mp4'
config_path = 'config/mgif-256.yaml'
checkpoint_path = './CNN2.pth.tar'
predict_mode = 'avd' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384


**Read image and video**

In [39]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)


source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [41]:
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

RuntimeError: Error(s) in loading state_dict for AVDNetwork:
	Missing key(s) in state_dict: "id_encoder.3.weight", "id_encoder.3.bias", "id_encoder.4.running_mean", "id_encoder.4.running_var", "id_encoder.6.weight", "id_encoder.6.bias", "id_encoder.7.weight", "id_encoder.7.bias", "id_encoder.7.running_mean", "id_encoder.7.running_var", "pose_encoder.3.weight", "pose_encoder.3.bias", "pose_encoder.4.running_mean", "pose_encoder.4.running_var", "pose_encoder.6.weight", "pose_encoder.6.bias", "pose_encoder.7.weight", "pose_encoder.7.bias", "pose_encoder.7.running_mean", "pose_encoder.7.running_var". 
	Unexpected key(s) in state_dict: "fc1.0.weight", "fc1.0.bias", "fc1.1.weight", "fc1.1.bias", "fc1.1.running_mean", "fc1.1.running_var", "fc1.1.num_batches_tracked", "fc1.3.weight", "fc1.3.bias", "fc2.0.weight", "fc2.0.bias", "fc2.1.weight", "fc2.1.bias", "fc2.1.running_mean", "fc2.1.running_var", "fc2.1.num_batches_tracked", "fc2.3.weight", "fc2.3.bias", "id_encoder.11.weight", "id_encoder.1

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 12/12 [00:16<00:00,  1.39s/it]
